In [1]:
import numpy as np
import MRzeroCore as mr0
import pypulseq as pp
import torch
import matplotlib.pyplot as plt
import matplotlib
from seq_utils import plot_signal
matplotlib.use('TkAgg')

In [6]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

experiment_id = 'exB09_GRE_EPI_2D'

# %% S1. SETUP sys

# choose the scanner limits
system = pp.Opts(
    max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s',
    rf_ringdown_time=20e-6, rf_dead_time=100e-6,
    adc_dead_time=20e-6, grad_raster_time=10e-6
)

# %% S2. DEFINE the sequence
seq = pp.Sequence()

partial_fourier = False
# Define FOV and resolution
# fov = 240e-3
fov = 220e-3

slice_thickness = 8e-3
Nread = 64    # frequency encoding steps/samples
Nphase = 64    # phase encoding steps/samples

# Define rf events
rf1, _, _ = pp.make_sinc_pulse(
    flip_angle=90 * np.pi / 180, duration=1e-3,
    slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
    system=system, return_gz=True
)

# Define other gradients and ADC events
adc_duration_OG = 0.25e-3 # @param {type: "slider", min: 0.25e-3, max: 10e-3, step: 0.05e-3}
a = int(system.adc_raster_time * Nread * 10**7)
b = int(system.grad_raster_time * 10**7)
c = int(adc_duration_OG * 10**7)
lcm_ab = abs(a * b) // np.gcd(a, b)
adc_raster_duration = (lcm_ab if round(c / lcm_ab) == 0 else round(c / lcm_ab) * lcm_ab) / 10**7

eddy_currents=True # @param {type:"boolean"}
eddy_currents_induced_delay= 0.0000015 # @param {type: "slider", min: -1e-4, max: 1e-4, step: 1e-8}
eddy_currents_induced_delay*=eddy_currents

gx = pp.make_trapezoid(channel='x', flat_area=Nread / fov, flat_time=adc_raster_duration, system=system)
gx_ = pp.make_trapezoid(channel='x', flat_area=-Nread / fov, flat_time=adc_raster_duration, system=system)
adc = pp.make_adc(num_samples=Nread, duration=adc_raster_duration, phase_offset=0 * np.pi / 180, delay=gx.rise_time + eddy_currents_induced_delay, system=system)
gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=1e-3, system=system)

# ======
# CONSTRUCT SEQUENCE
# ======
blip_duration=0.1e-3 # @param {type: "slider", min: 0.1e-3, max: 50e-3, step: 0.05e-3}
gp_blip = pp.make_trapezoid(channel='y', area=1/fov, duration=blip_duration, system=system)

print(f"pre y: {-Nphase//2 / fov}")
print(f"delta k: {1/fov}")
print(f"Ny: {Nphase}")
seq.add_block(rf1)
gp = pp.make_trapezoid(channel='y', area=-Nphase//2 / fov, duration=1e-3, system=system)
seq.add_block(gx_pre, gp)

if partial_fourier:
    for ii in range(0, int((Nphase//2) * 9/16)):
        seq.add_block(gx,adc)
        seq.add_block(gp_blip)
        seq.add_block(gx_,adc)
        seq.add_block(gp_blip)
else:
    for ii in range(0, Nphase//2):
        seq.add_block(gx,adc)
        seq.add_block(gp_blip)
        seq.add_block(gx_,adc)
        seq.add_block(gp_blip)
        
    
ok, error_report = seq.check_timing()
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed! Error listing follows:')
    # print(error_report)

seq.write("epi_testing_2.seq") 

pre y: -145.45454545454547
delta k: 4.545454545454546
Ny: 64
Timing check failed! Error listing follows:


'1ea488039917958bd8e98b60167ed120'

In [3]:
seq = mr0.Sequence.import_file("epi_testing_2.seq")
seq.plot_kspace_trajectory()

In [11]:
#quick 2D brain phantom sim and plot
signal = mr0.util.simulate_2d(seq)
seq.plot(plot_now=False)
mr0.util.insert_signal_plot(seq=seq, signal =signal.numpy())
plt.show()

DEPRECATED: util.simulate_2d will be removed in the future.
Use util.simulate() instead (together with util.load_phantom() if necessary
Calculating repetition 1 / 1 - done


In [12]:
# ======
# VISUALIZATION
# ======
# seq.plot(plot_now=False)
# mr0.util.insert_signal_plot(seq=seq, signal=signal.numpy())
# plt.show()
plot_signal(signal, Nphase, Nread, 1)
print("hello")

hello


In [ ]:
# %% S6: MR IMAGE RECON of signal ::: #####################################
fig = plt.figure(figsize=(10,2))  # fig.clf()

if partial_fourier:
    kspace_adc = torch.reshape((signal), (int(Nphase * 9/16), Nread)).clone().t()
else:
    kspace_adc = torch.reshape((signal), (Nphase, Nread)).clone().t()

kspace = kspace_adc

# add zeros to kspace
# kspace = torch.cat((kspace, torch.zeros((Nread, Nphase - kspace.shape[1]))), 1)
kspace[:,0::2] = torch.flip(kspace[:,0::2],[0] )[:,:]
# fftshift FFT fftshift

# add rows to 0 to kspace
# kspace = torch.cat((kspace, torch.zeros((Nread, 15))), 1)
# kspace = torch.cat((torch.zeros((Nread, 15)), kspace), 1)
spectrum = torch.fft.fftshift(kspace)
space = torch.fft.fft2(spectrum)
space = torch.fft.ifftshift(space)

plt.subplot(141)
plt.title('k-space')
mr0.util.imshow(np.abs(kspace.numpy()))
plt.subplot(142)
plt.title('log. k-space')
mr0.util.imshow(np.log(np.abs(kspace.numpy())))

plt.subplot(143)
plt.title('FFT-magnitude')
mr0.util.imshow(np.abs(space.numpy()))
plt.colorbar()
plt.subplot(144)
plt.title('FFT-phase')
mr0.util.imshow(np.angle(space.numpy()), vmin=-np.pi, vmax=np.pi)
plt.colorbar()

In [ ]:
kspace.shape
# convert to numpy:
kspace_np = kspace.numpy()
# add dims to kspace_np
kspace_np = np.expand_dims(kspace_np, axis=0)
from partial_fourier_recon import pocs_pf
kspace_recon = pocs_pf(kspace_np, 10)
# drop o and last dim from kspace_recon
kspace_recon = kspace_recon.squeeze()
kspace_recon.shape

In [ ]:
# # kspFull_kxkycz, kspZpad_kxkycz = partial_fourier_recon.pf_recon_pocs_ms2d(kspace, 10)
# # transpose the test data using torch
# kspace = torch.tensor(kspace)
# kspace = kspace.squeeze()
# spectrum = torch.fft.fftshift(kspace)
# space = torch.fft.fft2(spectrum)
# space = torch.fft.ifftshift(space)

plt.subplot(141)
plt.title('k-space')
mr0.util.imshow(np.abs(kspace_recon))
plt.subplot(142)
plt.title('log. k-space')
mr0.util.imshow(np.log(np.abs(kspace_recon)+1))

spectrum_recon = np.fft.fftshift(kspace_recon)
space_recon = np.fft.fft2(spectrum_recon)
space_recon = np.fft.ifftshift(space_recon)

plt.subplot(143)
plt.title('FFT-magnitude')
mr0.util.imshow(np.abs(space_recon))
plt.colorbar()
plt.subplot(144)
plt.title('FFT-phase')
mr0.util.imshow(np.angle(space_recon), vmin=-np.pi, vmax=np.pi)
plt.colorbar()